In [1]:
# Given rows with the same street name, house numbers should be in increasing order
def replace_non_increasing(street_group):
    increasing_value = None
    for i in range(len(street_group)):
        if increasing_value is None or street_group.iloc[i]['no_maison'] > increasing_value:
            increasing_value = street_group.iloc[i]['no_maison']
        else:
            # When house numbers decrease, set them to the last increasing house number
            street_group.at[street_group.index[i], 'no_maison'] = increasing_value
    return street_group

def house_number_clean(input_file):
    
    # Read the Excel file
    df = pd.read_excel(input_file)
    
    # Find the total number of rows
    total_rows = len(df)
    
    ### NON-NUMERIC HOUSE NUMBER CLEANING

    # Find the number of non-numeric values in the no_maison column before processing
    non_numeric_rows_before_processing = df['no_maison'].apply(lambda x: not str(x).isnumeric()).sum()

    # Replace all non-numeric values with the previous numeric value above, else set value to NaN
    df['no_maison'] = pd.to_numeric(df['no_maison'], errors='coerce')
    
    # Replace NaN and "·" values with the value above it in the no_maison column
    df['no_maison'].fillna(method='ffill', inplace=True)
    df['no_maison'].replace('·', method='ffill', inplace=True)
    
    ### NON-INCREASING HOUSE NUMBER CLEANING
    
    # Find the number of rows with non-increasing values for the same street name before processing
    non_increasing_rows_before_processing = df.groupby('nom_rue')['no_maison'].diff().lt(0).sum()

    # Invalid house number prefix and suffix handler
    # Invalid prefix example: (12,13,114) instead of (12,13,14) => 114 % 100 = 14
    # Invalid suffix example: (12,13,140) instead of (12,13,14) => 140 // 10 = 14
    last_street_name = None
    for index, row in df.iterrows():
        if pd.isnull(row['no_maison']):
            if(index == 0):
                df.at[index, 'no_maison'] = 1
            else:
                df.at[index, 'no_maison'] = df.at[index - 1, 'no_maison']

        if row['no_maison'] > 100:
            if(index !=0):
                if (df.at[index, 'no_maison'] // 10 >= df.at[index - 1, 'no_maison']):
                    df.at[index, 'no_maison'] = df.at[index, 'no_maison'] // 10 # handles most 1-digit suffixes
                elif(df.at[index, 'no_maison'] % 100 >= df.at[index - 1, 'no_maison']):
                    df.at[index, 'no_maison'] = row['no_maison'] % 100 #handles most 1-digit prefixes
                else:
                    df.at[index, 'no_maison'] = df.at[index - 1, 'no_maison'] #handles all other prefixes and suffixes

    # Find the number of non-numeric values in the no_maison column after processing
    non_numeric_rows = df['no_maison'].isnull().sum()
    
    # Remove all remaining non-increasing house numbers per street
    df = df.groupby('nom_rue').apply(replace_non_increasing).reset_index(drop=True)
            
    # Find the number of rows with non-increasing values in the no_maison column for the same street name
    non_increasing_rows = (df.groupby('nom_rue')['no_maison'].diff() < 0).sum() if 'no_maison' in df else 0

    # print(f"For {input_file} with {total_rows} rows: before processing we had {non_numeric_rows_before_processing} non-numeric values and {non_increasing_rows_before_processing} non-increasing values on the same street, after processing we get {non_numeric_rows} non-numeric values, {non_increasing_rows} non-increasing values on the same street, {non_increasing_rows/total_rows*100:.1f}% bad house numbers remain.")

    # Save the resulting Excel file
    output_file = input_file.replace('.xlsx', '_filtered_numbers.xlsx')
    output_file = output_file.replace('./recensements','./recensements_cleaned')
    df.to_excel(output_file, index=False)

# Example function usage
# input_file = './recensements/1832.xlsx'
# house_number_clean(input_file)

In [2]:
import os
import pandas as pd

value_counts = {}
folder_path = './recensements'
clean_folder_path = './recensements_cleaned'
file_list = [filename for filename in os.listdir(folder_path) if filename.endswith(".xlsx")]
number_of_files = len(file_list)

# Make sure rows have increasing house numbers in all excel files
print("cleaning files")
#remove_folder(clean_folder_path)
for i, filename in enumerate(file_list,start=1):
    print(f"{i} out of {number_of_files} files cleaned")
    file_path = os.path.join(folder_path, filename)
    house_number_clean(file_path)

# Iterate through each file in the folder
for filename in os.listdir(clean_folder_path):
        file_path = os.path.join(clean_folder_path, filename)
        # Read the Excel file
        df = pd.read_excel(file_path)
        
        # Get the second column
        column_values = df.iloc[:, 1]
        # Count the occurrences of each value in the second column
        for value in column_values:
            if value in value_counts:
                value_counts[value] += 1
            else:
                value_counts[value] = 1

# Convert the dictionary to a pandas DataFrame
result_df = pd.DataFrame(list(value_counts.items()), columns=['Value', 'Count'])

# Sort the DataFrame by the 'Count' column in descending order
result_df = result_df.sort_values('Count', ascending=False)

# Values to filter out
values_to_filter = ["pre","martherey","st jean","laurent","de bourg","francois","annee de naissance","pierre","rue du pont","chauerau","chailly","marteray","fre","halle","jn francois","derriere","id id","laurens","grand s jean","grand jean","devant","cheneau","cile de vant","st laurens","et","cile derriere","du marche","patud","francoise","gd st jean","hallede st laurent","haurent","grand fs jean","st laur","d halle","walle","jn jean","grand","paud","frand chene","cite derrier","moulins de pepinet","rue de martheray","cheneau bourg","marthony","marthorey","jennes","grand f jean","salud","de francois","du sont","cuchy","sejan","le pont","duz re","montee de st laurent","marthe","chemin de bourg","place du pont","marthery","f pierre","martherey e","dre","de pierre","la barre","grand fr jean","d etienne","valud","st fran","chavanne","ft pierre","grand sn jean","aisse","cite devriere","id .","luite dri","no de leur recepisse","cile derrier","flaurent","della barre","mererie","halle de laurent","du re","chaucrau .","l walle","marthere","theneau de bourg","e aisse","rue de francois","cete derriere","no des","pennes","marthoray","etienne","marberay","st laurant","cite derric","ctraz","chaucraie","bourge","ste pierre","theneau bourg","halle de s laurt","slaurent","petit gjean","montee de st monte","lite de vans","l hopital","rue du pre","de mercerie","pont","monorier","nerie","no es","monte st laurent","uve","grangeneuve","sdu","dean","ler","luchy","de st laurent","no de","grand sjean","tre","cite dessons","monbe clauron","rapaz","pre du marche","sallaz","grotte","monblesson","halle f laurent so","veuf","marherey","chaueran","st martin","martberty","challes st laurens","chaz","boston","plaurent","de marthenay","cite ederriere","petit - jean","hallede tlaurent","montoie","calaire","halle de f de","palude","laurant","villards","falle de st laurent","ane de bourg","g chene","no de leurs","pelit chene","cite de vans","st francoise","detienne","martherey .","bis"]

# Filtering out specific values
result_df_filtered = result_df[~result_df['Value'].isin(values_to_filter)]

#take top 100
dictionary = result_df_filtered.sort_values('Count', ascending=False)[0:100]

# Save the DataFrame to a CSV file
dictionary.to_csv("dictionary.csv", index=False)

cleaning files
1 out of 23 files cleaned
2 out of 23 files cleaned
3 out of 23 files cleaned
4 out of 23 files cleaned
5 out of 23 files cleaned
6 out of 23 files cleaned
7 out of 23 files cleaned
8 out of 23 files cleaned
9 out of 23 files cleaned
10 out of 23 files cleaned
11 out of 23 files cleaned
12 out of 23 files cleaned
13 out of 23 files cleaned
14 out of 23 files cleaned
15 out of 23 files cleaned
16 out of 23 files cleaned
17 out of 23 files cleaned
18 out of 23 files cleaned
19 out of 23 files cleaned
20 out of 23 files cleaned
21 out of 23 files cleaned
22 out of 23 files cleaned
23 out of 23 files cleaned


In [3]:
from Levenshtein import distance
import difflib

# Create an empty DataFrame to store the final result
final_result = pd.DataFrame(columns=['nom_rue', 'Suggested_Street_Name', 'Levenshtein_Distance'])

# Function to find the closest match from the dictionary
def find_closest_match(word, dictionary, levenshtein_weight=0.5, substring_weight=0.5):
    min_distance = float('inf')
    closest_keyword = None

    for keyword in dictionary['Value']:
        # Calculate normalized Levenshtein distance
        levenshtein_distance = distance(str(word), str(keyword)) / len(keyword)

        # Calculate longest matching substring
        substring_match = difflib.SequenceMatcher(None, str(word), str(keyword)).find_longest_match(0, len(word), 0, len(keyword))
        substring_distance = 1 - substring_match.size / max(len(word), len(keyword))

        # Combine the distances with weights
        combined_distance = levenshtein_weight * levenshtein_distance + substring_weight * substring_distance

        if combined_distance < min_distance:
            min_distance = combined_distance
            closest_keyword = keyword

    return closest_keyword, min_distance

# Iterate through each file in the folder and process street names
for filename in os.listdir(clean_folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(clean_folder_path, filename)
        df = pd.read_excel(file_path)
        print("we are at " + str(filename))

        # Add columns for closest dictionary street and Levenshtein distance
        df['Suggested_Street_Name'] = ''
        df['Levenshtein_Distance'] = 0

        for index, row in df.iterrows():
            street_name = row['nom_rue']
            if street_name in ("·"):
                if index > 0:  # Check if it's not the first row
                    street_name = df.loc[index - 1, 'nom_rue']
                    if street_name in ("·"):
                        print("Problem with dots is here")
                    # Otherwise, you can update the current row in the DataFrame
                    else:
                        df.loc[index, 'nom_rue'] = street_name
                else:
                    print("Problem with the first row having '·' ")

            closest_keyword, min_distance = find_closest_match(street_name, dictionary)
            df.at[index, 'Suggested_Street_Name'] = closest_keyword
            df.at[index, 'Levenshtein_Distance'] = min_distance

        # Append the required columns to the final result
        df_temp = df[['nom_rue', 'Suggested_Street_Name', 'Levenshtein_Distance']]
        final_result = final_result._append(df_temp, ignore_index=True)

# Sort the DataFrame by the 'Levenshtein_Distance' column
final_result.sort_values('Levenshtein_Distance', inplace=True)

final_result.to_excel("distances.xlsx", index=False)

we are at 1832_filtered_numbers.xlsx
we are at 1835_corrige_filtered_numbers.xlsx
we are at 1836_filtered_numbers.xlsx
we are at 1837_filtered_numbers.xlsx
we are at 1838_filtered_numbers.xlsx
we are at 1839_filtered_numbers.xlsx
we are at 1840_filtered_numbers.xlsx
we are at 1841_filtered_numbers.xlsx
we are at 1842_filtered_numbers.xlsx
we are at 1843_filtered_numbers.xlsx
we are at 1844_filtered_numbers.xlsx
we are at 1845_filtered_numbers.xlsx
we are at 1846_filtered_numbers.xlsx
we are at 1847_filtered_numbers.xlsx
we are at 1848_filtered_numbers.xlsx
we are at 1849_filtered_numbers.xlsx
we are at 1850_filtered_numbers.xlsx
we are at 1851_filtered_numbers.xlsx
we are at 1852_filtered_numbers.xlsx
we are at 1853_filtered_numbers.xlsx
we are at 1854_filtered_numbers.xlsx
we are at 1855_corrige_filtered_numbers.xlsx
we are at 1855_filtered_numbers.xlsx


In [4]:
#"fre" et "tre" deviennent "barre" plutot que "du pre"
final_result.loc[final_result['nom_rue'].isin(['fre', 'tre']), 'Suggested_Street_Name'] = 'du pre'
#"gs jean" devient "jean" plutot que "grand st jean"
final_result.loc[final_result['nom_rue'] == 'gs jean', 'Suggested_Street_Name'] = 'grand st jean'
#"veuve" devient "vennes" plutot que "grange veuve"
final_result.loc[final_result['nom_rue'] == 'veuve', 'Suggested_Street_Name'] = 'grange veuve'
#"chaz" devient "jean" plutot que "etraz"
final_result.loc[final_result['nom_rue'] == 'chaz', 'Suggested_Street_Name'] = 'etraz'
#"pieur" devient "palud" plutot que "st pierre"
final_result.loc[final_result['nom_rue'] == 'pieur', 'Suggested_Street_Name'] = 'st pierre'
#"rue" et "ruc" devient "barre" et "ouchy" mais devraient etre effacés
final_result = final_result[final_result['nom_rue'] != 'rue']
final_result = final_result[final_result['nom_rue'] != 'ruc']
# "boston" doit etre supprimé car il devrait etre mappé à "bosson" pas "bourg" mais bosson apparait pas dans le dictionnaire
final_result = final_result[final_result['nom_rue'] != 'boston']
# "et" et "id id" doit etre supprimé car il ne peut pas etre mappé a un nom de rue
final_result = final_result[final_result['nom_rue'] != 'et']
final_result = final_result[final_result['nom_rue'] != 'id id']

#"grand" devient "jean" mais devrait etre effacé
final_result = final_result[final_result['nom_rue'] != 'grand']
#"veuf" devient "jean" mais devrait etre "grange veuve"
final_result.loc[final_result['nom_rue'] == 'veuf', 'Suggested_Street_Name'] = 'grange veuve'

#"francois" devient "francs" plutot que "st francois"
final_result.loc[final_result['nom_rue'] == 'francois', 'Suggested_Street_Name'] = 'st francois'

#"halle" devient "chally" plutot que "l halle"
final_result.loc[final_result['nom_rue'] == 'halle', 'Suggested_Street_Name'] = 'l halle'

#"vichy" devient "ouchy" mais devrait etre enlevé (s'il n'apparait pas dans le dictionnaire)

final_result.to_excel("distances.xlsx", index=False)

In [5]:
# Print statistics of the number of rows per Levenshtein distance
statistics = final_result['Levenshtein_Distance'].value_counts().sort_index()
print("Top 10 Levenshtein minimum distance row counts")
print(statistics[0:10])

# Calculate and print the number of rows with Levenshtein distance <= 3
num_rows_levenshtein_3 = sum(final_result['Levenshtein_Distance'] <= 3)
print("Number of rows with Levenshtein distance <= 3:", num_rows_levenshtein_3)

# Calculate and print the total number of rows
total_rows = len(final_result)
print("Total number of rows:", total_rows)

# Print statistics of the values and occurrences of each suggested keyword
keyword_statistics = final_result['Suggested_Street_Name'].value_counts()

print("Top 10 suggested keyword values:")
print(keyword_statistics[0:10])

# Find the top 10 suggested street names
top_suggested_street_names = final_result['Suggested_Street_Name'].value_counts().head(10).index.tolist()

# Create a dictionary to store the top 10 original street names for each suggested street name
top_original_street_names = {}

# Iterate through each of the top 10 suggested street names
for suggested_street_name in top_suggested_street_names:
    filtered_data = final_result[final_result['Suggested_Street_Name'] == suggested_street_name]
    original_street_counts = filtered_data['nom_rue'].value_counts()
    top_original_streets = list(original_street_counts.head(3).items())
    top_original_street_names[suggested_street_name] = top_original_streets

# Print the results
for suggested_street_name, top_original_streets in top_original_street_names.items():
    print(f"For the suggested street name '{suggested_street_name}':")
    print("Top 3 original street names and their occurrences:")
    for i, (original_street, occurrences) in enumerate(top_original_streets, start=1):
        print(f"{i}. {original_street}, {occurrences}")
    print("----")


Top 10 Levenshtein minimum distance row counts
Levenshtein_Distance
0.000000    33579
0.052632        4
0.062500       62
0.066667       30
0.076923      204
            ...  
1.218182       20
1.291399        1
1.317391       20
1.345652        1
1.450000        1
Name: count, Length: 837, dtype: int64
Number of rows with Levenshtein distance <= 3: 95300
Total number of rows: 95300
Top 10 suggested keyword values:
Suggested_Street_Name
martheray               5178
bourg                   4702
st laurent              4484
du pre                  4274
grand st jean           4119
verdonnet               3542
challe de st laurent    3424
cite derriere           2805
st francois             2675
ouchy                   2490
Name: count, dtype: int64
For the suggested street name 'martheray':
Top 3 original street names and their occurrences:
1. martheray, 1477
2. martherey, 1328
3. marteray, 265
----
For the suggested street name 'bourg':
Top 3 original street names and their occurrences:

In [6]:
# For all files in recensements_cleaned, append Suggested_Street_Name value given the current nom_rue value and the distances.csv mapping if the distance value in distances.csv is <= 3

# Reading the 'final_result' dataframe and dropping duplicates
unique_distances = final_result.drop_duplicates(subset='nom_rue', keep='first')

clean_folder_path = './recensements_cleaned'

# Iterate through each file in the folder and process street names
for filename in os.listdir(clean_folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(clean_folder_path, filename)
        df = pd.read_excel(file_path)

        # Merge the 'df' dataframe with 'unique_distances' based on the 'nom_rue' column
        merged_df = pd.merge(df, unique_distances, on='nom_rue', how='left')
        
        # Keep only the rows where the 'Levenshtein_Distance' column value is <= 3
        merged_df = merged_df[merged_df['Levenshtein_Distance'] <= 3]
        
        # Save the resulting Excel file
        output_file = file_path.replace('.xlsx', '_merged.xlsx')
        output_file = output_file.replace('./recensements_cleaned', './recensements_merged')
        merged_df.to_excel(output_file, index=False)
        


In [1]:
import os
import pandas as pd
import numpy as np
from Levenshtein import distance

recensements_merged_path = './recensements_merged'

# Use list comprehension for file reading
file_names = [filename for filename in os.listdir(recensements_merged_path) if filename.endswith(".xlsx") and not filename.startswith("~$")]
dataframes = [pd.read_excel(os.path.join(recensements_merged_path, filename)) for filename in file_names]

# Define the columns for comparison
comparison_columns = ['proprietaire_nom', 'chef_prenom', 'chef_nom', 'chef_annee_naissance', 'epouse_nom', 'epouse_annee_naissance', 'enfants_dans_la_commune_prenom', 'enfants_annee_naissance', 'chef_origine', 'chef_vocation']

# Function for distance calculation
def calculate_distance(row1, row2):
    return np.sum([distance(str(row1[col]), str(row2[col])) for col in comparison_columns])

# Function for matching
def match_rows(args):
    index, row, file_to_match = args
    min_distance = float('inf')
    min_row = None
    for matching_index, matching_row in file_to_match.iterrows():
        distance_sum = calculate_distance(row, matching_row)
        if distance_sum < min_distance:
            min_distance = distance_sum
            min_row = matching_row
    return min_row, min_distance

In [2]:
matches = []


for index, row in dataframes[0].iterrows():
    current_match = []
    total_match_distance = 0
    print("row to match: ",row[['chef_prenom', 'chef_nom','chef_vocation']])
    
    for file_to_match in dataframes[1:]:
        result, file_match_distance = match_rows((index, row, file_to_match))
        #if(distance<non_address_distance_threshold):
        current_match.append(result)
        #print("match appended:", result[['chef_prenom', 'chef_nom','chef_vocation']])
        #print("match distance: ", file_match_distance)
        total_match_distance += file_match_distance
    
    matches.append([current_match,total_match_distance])

#print(matches)
        
# Convert the matches to a list
matches = [[match for match in match_group] for match_group in matches]

# Print the top 10 rows with the most matches
top_matches = sorted(enumerate(matches), key=lambda x: np.sum([calculate_distance(dataframes[0].iloc[x[0]], row) for row in x[1]]))[:10]
for index, match_group in top_matches:
    print(f"Row {index} has matches in the following files:")
    for i, match_row in enumerate(match_group):
        print(f"   {file_names[i]}: {match_row}")

# Print the list of files for each of the top 3 rows with the most matches
for index, match_group in top_matches[:3]:
    print(f"\nTop match for Row {index} is in {len(match_group)} files:")
    for i, match_row in enumerate(match_group):
        print(f"   {file_names[i]}")

row to match:  chef_prenom         jean
chef_nom         forster
chef_vocation    commise
Name: 0, dtype: object
match appended: chef_prenom      sigion
chef_nom         ferber
chef_vocation      sier
Name: 442, dtype: object
match distance:  30
match appended: chef_prenom        fanny
chef_nom         rossier
chef_vocation    modiste
Name: 3067, dtype: object
match distance:  27
match appended: chef_prenom         jean
chef_nom           kraer
chef_vocation    rentier
Name: 1452, dtype: object
match distance:  25
match appended: chef_prenom       jules
chef_nom         dirlam
chef_vocation    commis
Name: 2145, dtype: object
match distance:  26
match appended: chef_prenom       jean
chef_nom         kraer
chef_vocation        ·
Name: 647, dtype: object
match distance:  28
match appended: chef_prenom           jean
chef_nom         thuillard
chef_vocation     vigneron
Name: 385, dtype: object
match distance:  29
match appended: chef_prenom         jean
chef_nom           hofer
chef_voc

match appended: chef_prenom          jean
chef_nom           epring
chef_vocation    tailleur
Name: 2323, dtype: object
match distance:  31
match appended: chef_prenom       adolphe
chef_nom          reymond
chef_vocation    horloger
Name: 3804, dtype: object
match distance:  30
match appended: chef_prenom          jean
chef_nom           foring
chef_vocation    tailleur
Name: 1503, dtype: object
match distance:  31
match appended: chef_prenom            julid
chef_nom                 rod
chef_vocation    charpentier
Name: 684, dtype: object
match distance:  29
match appended: chef_prenom       louise
chef_nom          herrog
chef_vocation    regente
Name: 2447, dtype: object
match distance:  32
match appended: chef_prenom          henri
chef_nom            bonnet
chef_vocation    manoeuvre
Name: 1420, dtype: object
match distance:  33
match appended: chef_prenom         elisbel
chef_nom              blanc
chef_vocation    journalier
Name: 462, dtype: object
match distance:  32
match a

match appended: chef_prenom      jean francois
chef_nom                chavan
chef_vocation                ·
Name: 714, dtype: object
match distance:  37
row to match:  chef_prenom        benoite
chef_nom              rung
chef_vocation    parfemeur
Name: 5, dtype: object
match appended: chef_prenom       justin
chef_nom           andra
chef_vocation    rentier
Name: 470, dtype: object
match distance:  65
match appended: chef_prenom         henri
chef_nom            bally
chef_vocation    tourneur
Name: 1726, dtype: object
match distance:  60
match appended: chef_prenom         henry
chef_nom           corbaz
chef_vocation    vigneron
Name: 2544, dtype: object
match distance:  59
match appended: chef_prenom        samuel
chef_nom         seillens
chef_vocation     sirpier
Name: 3680, dtype: object
match distance:  57
match appended: chef_prenom      victor henri
chef_nom             farandoz
chef_vocation        tailleur
Name: 322, dtype: object
match distance:  61
match appended: chef

match appended: chef_prenom        francois
chef_nom         delapierre
chef_vocation          mauz
Name: 1125, dtype: object
match distance:  33
match appended: chef_prenom      francoise
chef_nom            dantan
chef_vocation       marche
Name: 1473, dtype: object
match distance:  31
match appended: chef_prenom      veuve francoise
chef_nom                   bovet
chef_vocation              entre
Name: 3342, dtype: object
match distance:  32
match appended: chef_prenom      francoise
chef_nom            dentan
chef_vocation    marchande
Name: 1571, dtype: object
match distance:  34
match appended: chef_prenom      alexandrine
chef_nom            dessalle
chef_vocation      marchande
Name: 1958, dtype: object
match distance:  33
match appended: chef_prenom      jn francoise
chef_nom              jaumine
chef_vocation         talione
Name: 2772, dtype: object
match distance:  33
match appended: chef_prenom      veuve marianne
chef_nom                 pettet
chef_vocation         blan

match appended: chef_prenom      vvede|pisse fs
chef_nom              marguerat
chef_vocation                 ·
Name: 2142, dtype: object
match distance:  24
row to match:  chef_prenom      charlotte
chef_nom          blanchet
chef_vocation            ·
Name: 10, dtype: object
match appended: chef_prenom       rosette
chef_nom         audibert
chef_vocation        piss
Name: 358, dtype: object
match distance:  22
match appended: chef_prenom      charlotte
chef_nom             carly
chef_vocation            ·
Name: 2250, dtype: object
match distance:  14
match appended: chef_prenom      isaline
chef_nom         meystre
chef_vocation          ·
Name: 165, dtype: object
match distance:  14
match appended: chef_prenom      charles
chef_nom               ·
chef_vocation          ·
Name: 1422, dtype: object
match distance:  13
match appended: chef_prenom      henriette
chef_nom             renou
chef_vocation            ·
Name: 3133, dtype: object
match distance:  13
match appended: chef_pre

match appended: chef_prenom      jean louis
chef_nom             corsat
chef_vocation        idmier
Name: 1943, dtype: object
match distance:  21
match appended: chef_prenom      jean samuel
chef_nom              deprez
chef_vocation       tionnier
Name: 3318, dtype: object
match distance:  22
match appended: chef_prenom      jean jaques
chef_nom               alder
chef_vocation            rex
Name: 1555, dtype: object
match distance:  21
match appended: chef_prenom      jean isaac
chef_nom                lin
chef_vocation         jennr
Name: 1149, dtype: object
match distance:  17
match appended: chef_prenom      jean marc
chef_nom            olivet
chef_vocation     tailleur
Name: 2813, dtype: object
match distance:  20
match appended: chef_prenom      jean justin
chef_nom               rosat
chef_vocation      jemnelier
Name: 3047, dtype: object
match distance:  21
match appended: chef_prenom      jean justin
chef_nom              corsat
chef_vocation              ·
Name: 744, dtyp

match appended: chef_prenom       francois
chef_nom             papon
chef_vocation    marchande
Name: 3083, dtype: object
match distance:  37
match appended: chef_prenom         louis
chef_nom          huprack
chef_vocation    marechal
Name: 3604, dtype: object
match distance:  37
match appended: chef_prenom         veuve
chef_nom           durand
chef_vocation    rentiere
Name: 563, dtype: object
match distance:  34
match appended: chef_prenom         isaac
chef_nom         gonthier
chef_vocation    marchand
Name: 2103, dtype: object
match distance:  36
match appended: chef_prenom      jeannette
chef_nom           ziegler
chef_vocation     marchand
Name: 3594, dtype: object
match distance:  38
match appended: chef_prenom      marie mord
chef_nom             fancon
chef_vocation     marchande
Name: 1351, dtype: object
match distance:  37
match appended: chef_prenom         veuve
chef_nom            piund
chef_vocation    laitiere
Name: 3047, dtype: object
match distance:  39
match app